Martech Challenge-Fauad Detection

In [ ]:
import os

import pandas as pd
import numpy as np
from paddle.io import Dataset
from baseline_tools import *

DATA_RATIO = 0.9  # 训练集和验证集比例

# None表示不使用，“emb”为Embedding预处理方案，选手可自由选择使用字段以及预处理方案
TAGS = {'android_id': None,
        'apptype': "emb",
        'carrier': "emb",
        'dev_height': "emb",
        'dev_ppi': "emb",
        'dev_width': "emb",
        'lan': "emb",
        'media_id': "emb",
        'ntt': "emb",
        'os': "emb",
        'osv': "emb",
        'package': "emb",
        'sid': None,
        'timestamp': "norm",
        'version': "emb",
        'fea_hash': "norm",
        'location': "emb",
        'fea1_hash': "norm",
        'cus_type': "emb"}

# 归一化权重设置
NORM_WEIGHT = {'timestamp': 6.40986e-13}

DATA_PATH = './data/'
TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')
SAVE_PATH = "./emb_dicts"

if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)
if not os.path.exists(SAVE_PATH):
    os.mkdir(SAVE_PATH)

In [ ]:
# 下载文件，在线Notebook可以不执行
import requests

train_url = 'https://bj.bcebos.com/v1/ai-studio-online/9da317c2532d40df8e362216b301817eb2cac8fbcfa647cb9fbbcfe1368eb314?responseContentDisposition=attachment%3B%20filename%3Dtrain.csv&authorization=bce-auth-v1%2F0ef6765c1e494918bc0d4c3ca3e5c6d1%2F2020-05-10T09%3A42%3A20Z%2F-1%2F%2F4976383af5ed6bfab656248934a640d32f998cf4b064389c094c362fb0fb6637'

test_url = 'https://bj.bcebos.com/v1/ai-studio-online/31bc052d3f0d45f3855ba783c4412847165b7a96b4464ba380d54c4f950aa182?responseContentDisposition=attachment%3B%20filename%3Dtest1.csv&authorization=bce-auth-v1%2F0ef6765c1e494918bc0d4c3ca3e5c6d1%2F2020-05-10T09%3A42%3A16Z%2F-1%2F%2F3af997efcf90abe9d0728b16d77a67462c8b4a8f96392d0e476e70fe3a6185fc'


train_path = './data/train.csv'
test_path = './data/test.csv'

if not os.path.exists(train_path):
    train_file = requests.get(train_url, allow_redirects=True)
    open(train_path, 'wb').write(train_file.content)

if not os.path.exists(test_path):
    test_file = requests.get(test_url, allow_redirects=True)
    open(test_path, 'wb').write(test_file.content)


In [ ]:
##我这里是对fea_hash，fea1_hash,timestamp这3个字段用norm处理，可是发现fea_hash数据集中有很多异常值，进行处理一下
#fea_hash字段中有许多异常数据，将他们改为最大值的一半：499997879.0
datas = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
#datas = datas["fea_hash"]

for ids,data in enumerate(datas["fea_hash"]):
    try:
        data = float(data)
    except:
        datas["fea_hash"][ids] = 499997879
        print(ids+1)
datas.to_csv("train.csv")
#fea_hash字段中有许多异常数据，将他们改为最大值的一半：499997879.0
datas = pd.read_csv(os.path.join(DATA_PATH, "test.csv"), dtype=str)
#datas = datas["fea_hash"]
#print(datas.head)

for ids,data in enumerate(datas["fea_hash"]):
    try:
        data = float(data)
    except:
        datas["fea_hash"][ids] = 499997879
        print(ids+1)
datas = datas
datas.to_csv("test.csv")


#fea_hash字段中有许多异常数据，将他们改为最大值的一半：499997879.0
datas = pd.read_csv(os.path.join(DATA_PATH, "mini_train.csv"), dtype=str)
#datas = datas["fea_hash"]
#print(datas.head)

for ids,data in enumerate(datas["fea_hash"]):
    try:
        data = float(data)
    except:
        datas["fea_hash"][ids] = 499997879
        print(ids+1)
datas = datas
datas.to_csv("mini_train.csv")

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


4215
6768
7743
10087
19697
24014
27327
34292
38562
50619
53075
55580
74299
75458
90843
96711
99606
107604
121318
122639
123620
124425
124601
128381
130010
130151
149210
160431
183562
192440
192949
195042
195558
196203
197781
199221
206751
208835
218310
221268
235515
236317
241150
244827
246415
246633
251970
260617
270554
272613
274158
276243
281199
284265
284534
297093
297217
299511
308107
328940
331027
331779
334515
349370
353159
356383
363213
370185
372572
380488
384243
402064
411594
414178
428177
430122
436263
441029
446758
449167
461534
469251
482030
482940
490806
494814
494836
495654
495874
499387
1118
2689
16438
22989
26666
29790
34043
37160
55882
69743
69997
80488
80965
88168
98768
109483
115193
121118
134953
136158
144610
147897
149259
4215
6768
7743
10087
19697


DAE

In [ ]:
df = pd.read_csv('train.csv', index_col=0)  # modified 

pack = dict()
for tag, tag_method in TAGS.items():
    if tag_method != "emb":
        if tag_method == "norm":
            data = df.loc[:, tag]
            print("{}_max的倒数:{}".format(tag,1/float(data.max())),"--------",float(data.max())/2)
            print("{}_max:{}".format(tag,float(data.max())),"--------min:",float(data.min()))
            #print("{}_mean:{}".format(tag,data.mean()))
        continue
    data = df.loc[:, tag]
    dict_size = make_dict_file(data, SAVE_PATH, dict_name=tag)
    pack[tag] = dict_size + 1  # +1是为了增加字典中不存在的情况，提供一个默认值

with open(os.path.join(SAVE_PATH, "size.dict"), "w", encoding="utf-8") as f:
    f.write(str(pack))

print("全部生成完毕")

apptype 字典生成完毕,共 89 个id
carrier 字典生成完毕,共 5 个id
dev_height 字典生成完毕,共 798 个id
dev_ppi 字典生成完毕,共 92 个id
dev_width 字典生成完毕,共 346 个id
lan 字典生成完毕,共 22 个id
media_id 字典生成完毕,共 284 个id
ntt 字典生成完毕,共 8 个id
os 字典生成完毕,共 2 个id
osv 字典生成完毕,共 155 个id
package 字典生成完毕,共 1950 个id
timestamp_max的倒数:6.409861939473897e-13 -------- 780048002158.7462
timestamp_max:1560096004317.4924 --------min: 1559491201174.7812
version 字典生成完毕,共 22 个id
fea_hash_max的倒数:2.3283201561138293e-10 -------- 2147470994.0
fea_hash_max:4294941988.0 --------min: 0.0
location 字典生成完毕,共 332 个id
fea1_hash_max的倒数:2.3299594677571534e-10 -------- 2145960077.5
fea1_hash_max:4291920155.0 --------min: 12400.0
cus_type 字典生成完毕,共 58 个id
全部生成完毕


setting norm weight

In [ ]:
# 归一化权重固定设置，这里用上面生成字典时候输出的数据
# 设置为最大值的一般
NORM_WEIGHT = {'timestamp': 6.409845522722902e-13,
                "fea_hash":2.3283201561138293e-10,
                "fea1_hash":2.3299594677571534e-10,
                "android_id":1.4086530e-06,
                "dev_height":0.00011081560283687943,
                "dev_ppi":0.001388888888888889,
                "dev_width":0.00011322463768115942
                }

In [ ]:
def get_size_dict(dict_path="./emb_dicts/size.dict"):
    """
    获取Embedding推荐大小
    :param dict_path: 由run_make_emb_dict.py生成的size.dict
    :return: 推荐大小字典{key: num}
    """
    with open(dict_path, "r", encoding="utf-8") as f:
        try:
            size_dict = eval(f.read())
        except Exception as e:
            print("size_dict打开失败，请检查", dict_path, "文件是否正常，报错信息如下:\n", e)
        return size_dict


class Reader(Dataset):
    def __init__(self,
                 is_infer: bool = False,
                 is_val: bool = False,
                 use_mini_train: bool = False,
                 emb_dict_path="./emb_dicts"):

        """
        数据读取类
        :param is_infer: 是否为预测Reader
        :param is_val: 是否为验证Reader
        :param use_mini_train：使用Mini数据集
        :param emb_dict_path: emb字典路径
        """
        super().__init__()
        # 选择文件名
        train_name = "mini_train" if use_mini_train else "train"
        file_name = "test" if is_infer else train_name
        # 根据文件名读取对应csv文件
        df = pd.read_csv(file_name + ".csv")
        # 划分数据集
        if is_infer:
            self.df = df.reset_index()
        else:
            start_index = 0 if not is_val else int(len(df) * DATA_RATIO)
            end_index = int(len(df) * DATA_RATIO) if not is_val else len(df)
            self.df = df.loc[start_index:end_index].reset_index()
        # 数据预处理
        self.cols = [tag for tag, tag_method in TAGS.items() if tag_method is not None]
        self.methods = dict()
        for col in self.cols:
            # ===== 预处理方法注册 =====
            if TAGS[col] == "emb":
                self.methods[col] = Data2IdEmb(dict_path=emb_dict_path, dict_name=col).get_method()
            elif TAGS[col] == "norm":
                self.methods[col] = Data2IdNorm(norm_weight=NORM_WEIGHT[col]).get_method()
            else:
                raise Exception(str(TAGS) + "是未知的预处理方案，请选手在此位置使用elif注册")

        # 设置FLAG负责控制__getitem__的pack是否包含label
        self.add_label = not is_infer
        # 设置FLAG负责控制数据集划分情况
        self.is_val = is_val

    def __getitem__(self, index):
        """
        获取sample
        :param index: sample_id
        :return: sample
        """
        # 因为本次数据集的字段非常多，这里就使用一个列表来"收纳"这些数据
        pack = []
        # 遍历指定数量的字段
        for col in self.cols:
            sample = self.df.loc[index, col]
            sample = self.methods[col](sample)
            pack.append(sample)

        # 如果不是预测，则添加标签数据
        if self.add_label:
            tag_data = self.df.loc[index, "label"]
            tag_data = np.array(tag_data).astype("int64")
            pack.append(tag_data)
            return pack
        else:
            return pack

    def __len__(self):
        return len(self.df)

In [ ]:
# 用于独立测试数据读取是否正常 推荐在本地IDE中下断点进行测试
print("检查数据ing...")
val_reader = Reader()
print(len(val_reader))
for data_id, data in enumerate(val_reader):
    for i in range(len(data)):
        if data_id == 0:
            print("第", i, "个字段 值为：", data[i])
        else:
            break
    if data_id % 1000 == 0:
        print("第", data_id, "条数据可正常读取 正在检查中", end="\r")
    if data_id == len(val_reader) - 1: 
        print("数据检查完毕")
        break

检查数据ing...
450001
第 0 个字段 值为： [1]
第 1 个字段 值为： [1]
第 2 个字段 值为： [1]
第 3 个字段 值为： [1]
第 4 个字段 值为： [1]
第 5 个字段 值为： [1]
第 6 个字段 值为： [1]
第 7 个字段 值为： [1]
第 8 个字段 值为： [1]
第 9 个字段 值为： [1]
第 10 个字段 值为： [1]
第 11 个字段 值为： [0.99986714]
第 12 个字段 值为： [1]
第 13 个字段 值为： [0.49710086]
第 14 个字段 值为： [1]
第 15 个字段 值为： [0.54280376]
第 16 个字段 值为： [1]
第 17 个字段 值为： 1
数据检查完毕00 条数据可正常读取 正在检查中


In [14]:
import os

import numpy as np
import pandas as pd
import paddle
import paddle.nn as nn
import paddle.tensor as tensor
from paddle.static import InputSpec
from paddle.metric import Accuracy

# 模型保存与加载文件夹
SAVE_DIR = "./output/"

# 部分训练超参数
EPOCHS = 15  # 训练多少个循环
TRAIN_BATCH_SIZE = 128  # mini_batch 大小
EMB_SIZE = 128  # Embedding特征大小
EMB_LINEAR_SIZE = 16  # Embedding后接Linear层神经元数量
LINEAR_LAYERS_NUM = 1  # 归一化方案的Linear层数量
LEARNING_RATE = 0.0007  # 学习率


# 配置训练环境
USE_MINI_DATA = False  # 默认使用小数据集，此方法可加快模型产出速度，但可能会影响准确率

# 组网
class SampleNet(paddle.nn.Layer):
    def __init__(self, tag_dict: dict, size_dict: dict):
        # 继承Model
        super().__init__()
        # 新建一个隐藏层列表，用于存储各字段隐藏层对象
        self.hidden_layers_list = []
        # 定义一个用于记录输出层的输入大小变量，经过一个emb的网络结构就增加该结构的output_dim，以此类推
        out_layer_input_size = 0
        # 定义一个李斯特用来装处理的方式
        self.tag_method_list = []
        # 遍历每个字段以及其处理方式
        for tag, tag_method in tag_dict.items():
            # ===== 网络结构方法注册 =====
            # Embedding方法注册
            if tag_method == "emb":
                hidden_layer = nn.LayerList([
                    nn.Embedding(num_embeddings=size_dict[tag],embedding_dim=EMB_SIZE),
                    ])
                out_layer_input_size += EMB_LINEAR_SIZE
                self.tag_method_list.append('emb')
            # 归一化方法注册
            elif tag_method == "norm":
                hidden_layer = nn.LayerList([
                    nn.Linear(in_features=1, out_features=3),
                    nn.LeakyReLU(),
                    nn.Linear(in_features=3, out_features=1),
                    nn.LeakyReLU()
                    ])
                out_layer_input_size += 1
                self.tag_method_list.append('norm')
            # 如果对应方法为None，那么跳过该字段
            elif tag_method is None:
                continue
            # 若出现没有注册的方法，提示报错
            else:
                raise Exception(str(tag_method) + "为未知的处理方案，请在SampleNet类中用elif注册处理流程")
            self.hidden_layers_list.append(hidden_layer)
        # emb的特殊处理
        self.emb_lstm = nn.LSTM(EMB_SIZE, 128, 1)
        self.emb_fc = nn.Linear(in_features=128, out_features=EMB_LINEAR_SIZE)
        self.emb_relu = nn.LeakyReLU()
        # 定义输出层
        self.out_layers = nn.Linear(in_features=out_layer_input_size, out_features=42)
        self.out_layers1 = nn.Linear(in_features=42, out_features=2)
        self.dp_relu = nn.LeakyReLU()
        self.dp_out = nn.Dropout(0.2)

    def forward(self, *input_data):
        layer_list = []  # 用于存储各字段特征结果
        for sample_data, hidden_layers, tag_method in zip(input_data, self.hidden_layers_list, self.tag_method_list):
            tmp = sample_data
            for hidden_layer in hidden_layers:
                tmp = hidden_layer(tmp)  # 所有类型的特征都需要走对应的初始化方法
                if tag_method == 'norm':  # norm 的处理方式已经在前面写完了
                    pass
                elif tag_method == 'emb':
                    tmp, (_, _) = self.emb_lstm(tmp)
                    tmp = self.emb_fc(tmp)
                    tmp = self.emb_relu(tmp)

            layer_list.append(tensor.flatten(tmp, start_axis=1))  # flatten是因为原始shape为[batch size, 1 , *n], 需要变换为[bs, n]
        # 对所有字段的特征合并
        layers = tensor.concat(layer_list, axis=1)
        # 把特征放入用于输出层的网络
        result = self.out_layers(layers)
        result = self.out_layers1(result)

        result = paddle.nn.functional.softmax(result)
        # 返回分类结果
        return result


# 定义网络输入
inputs = []
for tag_name, tag_m in TAGS.items():
    d_type = "float32"
    if tag_m == "emb":
        d_type = "int64"
    if tag_m is None:
        continue
    inputs.append(InputSpec(shape=[-1, 1], dtype=d_type, name=tag_name))

In [11]:
# 定义Label
labels = [InputSpec([-1, 1], 'int64', name='label')]
# 实例化SampleNet
model = paddle.Model(SampleNet(TAGS, get_size_dict()), inputs=inputs, labels=labels)
# 获取训练集和测试集数据读取器
train_reader = Reader(use_mini_train=USE_MINI_DATA)
val_reader = Reader(use_mini_train=USE_MINI_DATA, is_val=True)
# 定义优化器
optimizer = paddle.optimizer.Adam(learning_rate=LEARNING_RATE, parameters=model.parameters())
# 模型训练配置
model.prepare(optimizer, paddle.nn.loss.CrossEntropyLoss(), Accuracy())
# 开始训练
model.fit(train_data=train_reader,  # 训练集数据
            eval_data=val_reader,  # 交叉验证集数据
            batch_size=TRAIN_BATCH_SIZE,  # Batch size大小
            epochs=EPOCHS,  # 训练轮数
            log_freq=1000,  # 日志打印间隔
            save_dir=SAVE_DIR)  # checkpoint保存路径

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/15


In [13]:
CHECK_POINT_ID = "0"  # 如没有训练完毕，可以挑选SAVE_DIR路径的下的中的checkpoint文件名(不包含拓展名哦)，例如"1"
TEST_BATCH_SIZE = 32  # 若因内存/显存发生报错，请优先调整为1
RESULT_FILE = "./result.csv"  # 推理文件保存位置

# 实例化SampleNet
model = paddle.Model(SampleNet(TAGS, get_size_dict()), inputs=inputs)
# 获取推理Reader并读取参数进行推理
infer_reader = Reader(is_infer=True)
model.load(os.path.join(SAVE_DIR, CHECK_POINT_ID))
# 开始推理
model.prepare()
infer_output = model.predict(infer_reader, TEST_BATCH_SIZE)
# 获取原始表中的字段并添加推理结果
result_df = infer_reader.df.loc[:, "sid"]
pack = []
for batch_out in infer_output[0]:
    for sample in batch_out:
        pack.append(np.argmax(sample))
# 保存csv文件
result_df = pd.DataFrame({"sid": np.array(result_df, dtype="int64"), "label": pack})
result_df.to_csv(RESULT_FILE, index=False)
print("结果文件保存至：", RESULT_FILE)

Predict begin...
step   14/4688 [..............................] - ETA: 2:12 - 28ms/st2201 不在 package 字典内, 由默认值代替，可能影响效果
step   20/4688 [..............................] - ETA: 2:09 - 28ms/st2025 不在 package 字典内, 由默认值代替，可能影响效果
step   22/4688 [..............................] - ETA: 2:08 - 28ms/step2336.0 不在 dev_width 字典内, 由默认值代替，可能影响效果
step   26/4688 [..............................] - ETA: 2:07 - 27ms/ste1249 不在 package 字典内, 由默认值代替，可能影响效果
1804 不在 package 字典内, 由默认值代替，可能影响效果
step   42/4688 [..............................] - ETA: 2:03 - 27ms/ste916.0 不在 dev_height 字典内, 由默认值代替，可能影响效果
step   54/4688 [..............................] - ETA: 2:01 - 26ms/ste1811 不在 package 字典内, 由默认值代替，可能影响效果
step   66/4688 [..............................] - ETA: 2:00 - 26ms/ste1174.0 不在 dev_height 字典内, 由默认值代替，可能影响效果
step   84/4688 [..............................] - ETA: 1:59 - 26ms/st150.0 不在 dev_width 字典内, 由默认值代替，可能影响效果
step   96/4688 [..............................] - ETA: 1:59 - 26ms/ste666.0 不在 dev_height 字典内,